## Numba + CUDA on Google Colab

By default, Google Colab is not able to run numba + CUDA, because two lilbraries are not found, `libdevice` and `libnvvm.so`. So we need to make sure that these libraries are found in the notebook.  

First, we look for these libraries on the system. To do that, we simply run the `find` command, to recursively look for these libraries starting at the root of the filesystem. The exclamation mark escapes the line so that it's executed by the Linux shell, and not by the jupyter notebook. 

In [70]:
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'


/usr/local/cuda-10.0/nvvm/libdevice
/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so


Then, we add the two libraries to numba environment variables:

In [0]:
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-10.0/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so"


And we're done! Now let's get started. 

## A first CUDA kernel

Let's get started by implementing a very simple CUDA kernel to compute the square root of each value in an array. First, here is our array: 

In [72]:
import numpy as np
a = np.arange(4096,dtype=np.float32)
a

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 4.093e+03, 4.094e+03,
       4.095e+03], dtype=float32)

As we have seen in [part II](https://thedatafrog.com/boost-python-gpu/), and as discussed in the introduction, we can simply use numba's vectorize decorator to compute the square root of all elements in parallel on the GPU: 

In [73]:
import math
from numba import vectorize

@vectorize(['float32(float32)'], target='cuda')
def gpu_sqrt(x):
    return math.sqrt(x)
  
gpu_sqrt(a)

array([ 0.       ,  1.       ,  1.4142135, ..., 63.97656  , 63.98437  ,
       63.992188 ], dtype=float32)

This time, as an exercise, we'll do the same with a custom CUDA kernel. 

We first define our kernel: 

In [0]:
from numba import cuda

@cuda.jit
def gpu_sqrt_kernel(x, out):
  idx = cuda.grid(1)
  out[idx] = math.sqrt(x[idx])

Let's discuss this code in some details. 

We have an input array of 4096 values, so we will use 4096 threads on the GPU. 

Our input and output arrays are one dimensional, so we will use a one-dimensional *grid* of threads (we will discuss grids in details in the next section). The call `cuda.grid(1)` returns the unique index for the current thread in the whole grid.  With 4096 threads, `idx` will range from 0 to 4095. 

Then, we see in the code that each thread is going to deal with a single element of the input array to produce a single element in the output array. This element is determined for each thread by the thread index. 

Now that we have our kernel, we copy our input array to the GPU device, create an output array on the device with the same shape, and finally launch the kernel: 

In [0]:
# move input data to the device
d_a = cuda.to_device(a)
# create output data on the device
d_out = cuda.device_array_like(d_a)

# we decide to use 32 blocks, each containing 128 threads
blocks_per_grid = 32
threads_per_block = 128
gpu_sqrt_kernel[blocks_per_grid, threads_per_block](d_a, d_out)
# wait for all threads to complete
cuda.synchronize()
# copy the output array back to the host system
# and print it
print(d_out.copy_to_host())

[ 0.         1.         1.4142135 ... 63.97656   63.98437   63.992188 ]


In the code above, the 4096 threads are arranged into a grid of 32 *blocks*, each block having 128 threads. This *execution configuration* will be discussed in the next section. 

**Exercises:**

- Go back to the previous cell, and try to decrease the number of blocks per grid, or the number of threads per block. 
- Then try to increase the number of blocks per grid, or the number of threads per blocks
- Try to remove the `cuda.synchronize()` call

**Results**: 

- When you reduce the number of threads, either by decreasing the number of blocks per grid or the number of threads per block, some elements are not processed, and the corresponding slots at the end of the output array remain set to their default value, which is 0. 
- If, on the other hand, you increase the number of threads, it seems that everything is working fine. However, this actually creates an error even though we cannot see it. We will see later how to expose this error. Debugging code is one of the difficulties of CUDA, as the error messages are not always visible.
- Finally, you might have expected that commenting out the call to `cuda.synchronize` would have resulted in output array only partially filled, or not filled at all. That would be a good guess since the CPU keeps running the main program (the one of the cell) while the GPU processes the data asynchronously. However, the copy to the host performs an implicit synchronization, so the call to cuda.syncronize is not necessary.  

## Execution configuration : number of blocks and number of threads

In our first example above, we decided to use 4096 threads. These threads were arranged as a grid containing 32 blocks of 128 threads each. 

You probably wondered why we decided to do that. So in this section, I will try and answer some of the questions you may have: 

- Why do we need to arrange threads into blocks within the grid? 
- How did we come up with the magical numbers 32 and 128? Could we use other values like 16 and 256 (which still total 4096), or 64 and 64? 

These questions are important if you care about performance -- and you probably do if you're here to learn how to speed up your calculations on a GPU! To answer these questions, we first need to learn about our GPU hardware. 



### CUDA: Learn about your GPU hardware

Let's find out about the GPU we are using. Please note that your GPU (or the GPU attributed to you on Google Colab) could be different from mine. Of course, the numbers I'm giving below and the picture are only valid for the GPU attributed to me during my session on Google Colab. 

In [0]:
cuda.detect()

Found 1 CUDA devices
id 0             b'Tesla T4'                              [SUPPORTED]
                      compute capability: 7.5
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

We see that's an nvidia [T4](https://www.nvidia.com/en-us/data-center/tesla-t4/), and here is the [white paper with the specs](https://www.nvidia.com/content/dam/en-zz/Solutions/design-visualization/technologies/turing-architecture/NVIDIA-Turing-Architecture-Whitepaper.pdf) for the Turing architecture. In this paper, we see that the T4 is based on the Turing TU102 GPU which has: 

- 72 *streaming multiprocessors* 
- 64 cuda cores per streaming multiprocessor

Here is a block diagram of the TU102 GPU. The streaming multiprocessors are the 72 inner blocks labelled SM.

![TU102 GPU](https://raw.githubusercontent.com/cbernet/maldives/master/numba/tu_102_diagram.jpg)



The [streaming multiprocessors](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#hardware-implementation) are the fundamental processing units of CUDA enabled GPUs. Each SM can execute hundreds of threads in parallel, and has all the resources needed to manage these threads. 

### Number of blocks and number of threads per block

When the kernel is launched, each block of threads is sent to a single SM, and each SM can end up with multiple blocks to process. 

The first conclusion we can draw is that if the number of blocks is lower than the number of SMs in the GPU, some of the SMs will remain idle. That was the case in our first simple example above: we used 32 blocks, and we have 72 SMs on the Tesla T4. Therefore, 40 SMs were not used. 

To process a block, the SM partitions it into groups of 32 threads called *warps*. A warp executes one common instruction at a given time in parallel for all threads in the warp. So full efficiency is realized if all warps in the block are complete. Therefore, the number of threads per block needs to be a multiple of 32. Moreover, to limit latency, this number should not be too low. 

Deciding which execution configuration to use is not easy, and the choice should be driven by performance analysis. However, here are some basic rules to get started: 

- **The number of blocks in the grid should be larger than the number of SMs on the GPU, typically 2 to 4 times larger.** 
- **The number of threads per block should be a multiple of 32, typically between 128 and 512.**

The Tesla T4 has 72 streaming multiprocessors. On paper, it will perform best on data arrays with a size larger than 74 * 2 * 128 ~ 20 000. And this is the bare minimum: this card will typically be used on arrays much larger than that. In our simple example above, we used an array of size 4096, which is way too small.

On the other hand, what if the data array is very large? Let's consider an array with one billion entries. Following the second rule above, we can choose a number of threads per block of 256, and we end up with 1e9 / 256 ~ 3 million blocks. Since the CUDA kernel launch overhead increases with the number of blocks, going for such a large number of blocks would seriously hit performance. In the following, we will see how to use striding to solve this problem. 

### Striding 

This simple kernel deals with a single element of the input array:

In [0]:
@cuda.jit
def gpu_atan(x, out):
  idx = cuda.grid(1)
  out[idx] = math.atan(x[idx])

When the kernel is deployed, the GPU therefore needs to create as many threads as elements in the array, which potentially results in many blocks if the array is large. 

On the contrary, a striding kernel deals with several elements of the input array, using a loop: 

In [0]:
@cuda.jit
def gpu_atan_stride(x, out):
  start = cuda.grid(1)
  stride = cuda.gridsize(1)
  for i in range(start, x.shape[0], stride): 
    out[i] = math.atan(x[i])

In this way, a given thread deals with several elements, and the number of threads is kept under control. Threads keep doing work in a coordinated way, and the GPU is not wasting time creating and scheduling threads. 

To understand the code above, let's consider a small example with: 

- an input data array of size 8
- blocks with 4 threads each

Without striding, we end up with two blocks, and 8 threads in total, each dealing with a single element in the array. 

With striding, we get one block, and 4 threads in total. Each thread deals with two elements.  Specifically, in the code above: 

- `start` is the thread index, which is 0, 1, 2, 3 for the four threads, respectively. 
- `stride` is the size of the grid, which is the total number of threads in the grid, here 4. 
- `x.shape[0]` is the size of the input data array `x`, which is 1D. So it's 8. 

So here are the elements processed by each thread: 

In [79]:
for thread_index in range(4):
  print('thread', thread_index)
  for j in range(thread_index, 8, 4):
    print('\telement', j)

thread 0
	element 0
	element 4
thread 1
	element 1
	element 5
thread 2
	element 2
	element 6
thread 3
	element 3
	element 7


### Striding on large datasets 



In [0]:
@cuda.jit
def gpu_atan_kernel(x, out):
  idx = cuda.grid(1)
  out[idx] = math.atan(x[idx])

In [0]:
import numpy as np
a = np.arange(100000,dtype=np.float32)
# move input data to the device
d_a = cuda.to_device(a)
# create output data on the device
d_out = cuda.device_array_like(d_a)

First, let's see how fast we can process this array sequentially (in a single thread) on the GPU:

In [0]:
%time gpu_atan_kernel[1, 1](d_a, d_out); cuda.synchronize()

CPU times: user 0 ns, sys: 893 µs, total: 893 µs
Wall time: 863 µs


In [0]:
%time gpu_sqrt_kernel[128,128](d_a, d_out); cuda.synchronize()

CPU times: user 1.17 ms, sys: 516 µs, total: 1.68 ms
Wall time: 3.78 ms


In [0]:


from math import hypot

@cuda.jit
def hypot_stride(a, b, out):
  idx = cuda.grid(1) 
  out[idx] = hypot(a[idx], b[idx])
  
# You do not need to modify the contents in this cell
n = 1000000
a = np.random.uniform(-12, 12, n).astype(np.float32)
b = np.random.uniform(-12, 12, n).astype(np.float32)
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.device_array_like(d_b)

blocks = 128
threads_per_block = 64

hypot_stride[blocks, threads_per_block](d_a, d_b, d_c)

In [0]:
%time hypot_stride[1,1](d_a, d_b, d_c); cuda.synchronize()

CPU times: user 1.06 ms, sys: 20 µs, total: 1.08 ms
Wall time: 3.46 ms


In [0]:
%time hypot_stride[128,64](d_a, d_b, d_c); cuda.synchronize()

CPU times: user 852 µs, sys: 0 ns, total: 852 µs
Wall time: 966 µs


In [0]:
@cuda.jit
def gpu_atan_nostride(x, out):
  idx = cuda.grid(1)
  out[idx] = math.atan(x[idx])

In [0]:
n = 10000000
a = np.random.uniform(-5, 5, n).astype(np.float32)
d_a = cuda.to_device(a)
d_out = cuda.device_array_like(d_a)

In [53]:
%time gpu_atan_stride[1,1](d_a, d_out), cuda.synchronize()


CPU times: user 1.37 s, sys: 1.02 s, total: 2.39 s
Wall time: 2.4 s


(None, None)

In [54]:
%time gpu_atan_stride[128,64](d_a, d_out), cuda.synchronize()


CPU times: user 686 µs, sys: 1.85 ms, total: 2.53 ms
Wall time: 2 ms


(None, None)

In [69]:
%time gpu_atan_nostride[1,1](d_a, d_out), cuda.synchronize()


CPU times: user 127 ms, sys: 1.55 ms, total: 129 ms
Wall time: 130 ms


(None, None)

In [64]:
%time gpu_atan_nostride[128,64](d_a, d_out), cuda.synchronize()


CPU times: user 1.65 ms, sys: 0 ns, total: 1.65 ms
Wall time: 3.05 ms


(None, None)